# colab test

In [1]:
%load_ext autoreload
%autoreload 2

In [24]:
import pandas as pd
import bert_ner
import csv
import torch
from transformers import BertForTokenClassification, BertTokenizer
from tqdm.notebook import tqdm

MAX_LEN = 256
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 2
EPOCHS = 40
LEARNING_RATE = 1e-05
MAX_GRAD_NORM = 10


path = "Train_Tagged_Titles.tsv"
df = pd.read_csv(path, sep="\t", dtype=str, keep_default_na=False, na_values=[""], quoting=csv.QUOTE_NONE)
df["Record Number"] = df["Record Number"].astype(int)
# forward fill NaN values with last valid value
df = df.ffill()
train_df = bert_ner.convert_df(df)

tokenizer = BertTokenizer.from_pretrained('bert-base-german-cased')

train_dl, labels2id, id2labels, train_ds = bert_ner.create_dl(train_df, df, tokenizer)

device = torch.device('cpu')

model = BertForTokenClassification.from_pretrained(
'bert-base-german-cased', num_labels=len(labels2id), id2label=id2labels, label2id=labels2id)

model.to(device)

optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE)

pbar = tqdm(range(EPOCHS), desc="Loss: N/A")
for e in enumerate(pbar):
    pbar.set_description(f'Loss: f{bert_ner.training_loop(model, train_dl, optimizer, device)}')
    if (e + 1) % 10 == 0:
        torch.save({'model_state_dict': model.state_dict(), 'optim_state_dict': optim.state_dict()}, f'bert_german_0_e{e + 1}.p')

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-german-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Loss: N/A:   0%|          | 0/40 [00:00<?, ?it/s]


loss: n/a:   0%|          | 0/2500 [00:00<?, ?it/s]

Training loss per 100 training steps: 3.9368972778320312



loss: 3.1353261470794678:   0%|          | 4/2500 [00:17<3:03:55,  4.42s/it]


KeyboardInterrupt: 